Import necessary libraries

In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
import tensorflow as tf
from tensorflow import keras
import numpy as np
from sklearn.metrics import accuracy_score, roc_auc_score, f1_score
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import precision_recall_curve, f1_score, roc_auc_score, accuracy_score, confusion_matrix, classification_report

### Understand the Data

In [2]:
df = pd.read_csv('healthy_eating_dataset.csv')

print(df.info())
print("----------------------------------------------")

print(df.describe())
print("----------------------------------------------")

print(df.isnull().sum())
print("----------------------------------------------")

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2000 entries, 0 to 1999
Data columns (total 20 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   meal_id         2000 non-null   int64  
 1   meal_name       2000 non-null   object 
 2   cuisine         2000 non-null   object 
 3   meal_type       2000 non-null   object 
 4   diet_type       2000 non-null   object 
 5   calories        2000 non-null   int64  
 6   protein_g       2000 non-null   float64
 7   carbs_g         2000 non-null   float64
 8   fat_g           2000 non-null   float64
 9   fiber_g         2000 non-null   float64
 10  sugar_g         2000 non-null   float64
 11  sodium_mg       2000 non-null   int64  
 12  cholesterol_mg  2000 non-null   int64  
 13  serving_size_g  2000 non-null   int64  
 14  cooking_method  2000 non-null   object 
 15  prep_time_min   2000 non-null   int64  
 16  cook_time_min   2000 non-null   int64  
 17  rating          2000 non-null   f

In [3]:
df.head()

,meal_id,meal_name,cuisine,meal_type,diet_type,calories,protein_g,carbs_g,fat_g,fiber_g,sugar_g,sodium_mg,cholesterol_mg,serving_size_g,cooking_method,prep_time_min,cook_time_min,rating,is_healthy,image_url
0,1,Kid Pasta,Indian,Lunch,Keto,737,52.4,43.9,34.3,16.8,42.9,2079,91,206,Grilled,47,56,4.4,0,https://example.com/images/meal_1.jpg
1,2,Husband Rice,Mexican,Lunch,Paleo,182,74.7,144.4,0.1,22.3,38.6,423,7,317,Roasted,51,34,2.4,0,https://example.com/images/meal_2.jpg
2,3,Activity Rice,Indian,Snack,Paleo,881,52.9,97.3,18.8,20.0,37.5,2383,209,395,Boiled,58,29,4.3,0,https://example.com/images/meal_3.jpg
3,4,Another Salad,Mexican,Snack,Keto,427,17.5,73.1,7.6,9.8,41.7,846,107,499,Grilled,14,81,4.6,0,https://example.com/images/meal_4.jpg
4,5,Quite Stew,Thai,Lunch,Vegan,210,51.6,104.3,26.3,24.8,18.2,1460,42,486,Raw,47,105,4.3,0,https://example.com/images/meal_5.jpg


In [4]:
df.columns

Index(['meal_id', 'meal_name', 'cuisine', 'meal_type', 'diet_type', 'calories',
       'protein_g', 'carbs_g', 'fat_g', 'fiber_g', 'sugar_g', 'sodium_mg',
       'cholesterol_mg', 'serving_size_g', 'cooking_method', 'prep_time_min',
       'cook_time_min', 'rating', 'is_healthy', 'image_url'],
      dtype='object')

In [5]:
len(df['meal_name'].unique())

1750

In [6]:
df['cuisine'].unique()

array(['Indian', 'Mexican', 'Thai', 'Italian', 'American', 'Chinese',
       'Mediterranean', 'Japanese'], dtype=object)

In [7]:
df['diet_type'].unique()

array(['Keto', 'Paleo', 'Vegan', 'Balanced', 'Vegetarian', 'Low-Carb'],
      dtype=object)

In [8]:
df['cooking_method'].unique()

array(['Grilled', 'Roasted', 'Boiled', 'Raw', 'Steamed', 'Baked', 'Fried'],
      dtype=object)

In [9]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2000 entries, 0 to 1999
Data columns (total 20 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   meal_id         2000 non-null   int64  
 1   meal_name       2000 non-null   object 
 2   cuisine         2000 non-null   object 
 3   meal_type       2000 non-null   object 
 4   diet_type       2000 non-null   object 
 5   calories        2000 non-null   int64  
 6   protein_g       2000 non-null   float64
 7   carbs_g         2000 non-null   float64
 8   fat_g           2000 non-null   float64
 9   fiber_g         2000 non-null   float64
 10  sugar_g         2000 non-null   float64
 11  sodium_mg       2000 non-null   int64  
 12  cholesterol_mg  2000 non-null   int64  
 13  serving_size_g  2000 non-null   int64  
 14  cooking_method  2000 non-null   object 
 15  prep_time_min   2000 non-null   int64  
 16  cook_time_min   2000 non-null   int64  
 17  rating          2000 non-null   f

### Data Preprocessing
Feature Engineering, One-Hot-Encoding, Outlier Cleaning

In [10]:
import numpy as np
import pandas as pd

# control outliers with clip_quantiles
def engineer_meal_features(df: pd.DataFrame, clip_quantiles=(0.01, 0.99)):
    out = df.copy()

    # Drop obvious non-features
    drop_cols = ["meal_id", "meal_name", "image_url"]
    out = out.drop(columns=[c for c in drop_cols if c in out.columns])

    # Time features
    out["total_time_min"] = out["prep_time_min"] + out["cook_time_min"]
    out["is_quick_<=20min"] = (out["total_time_min"] <= 20).astype(int)

    # Macro kcals
    out["kcal_from_protein"] = out["protein_g"] * 4.0
    out["kcal_from_carbs"]   = out["carbs_g"]   * 4.0
    out["kcal_from_fat"]     = out["fat_g"]     * 9.0
    out["macro_kcal_sum"]    = out["kcal_from_protein"] + out["kcal_from_carbs"] + out["kcal_from_fat"]
    out["macro_kcal_gap"]    = out["calories"] - out["macro_kcal_sum"]

    # Macro % shares 
    # clip(0, 1) ensures no out of bounds percentages
    denom = out["calories"]
    out["protein_pct"] = (out["kcal_from_protein"] / denom).clip(0, 1)
    out["carb_pct"]    = (out["kcal_from_carbs"]   / denom).clip(0, 1)
    out["fat_pct"]     = (out["kcal_from_fat"]     / denom).clip(0, 1)
    
    # Per-100kcal densities
    kcal = out["calories"]
    out["sugar_per_100kcal"]   = (out["sugar_g"]   / (kcal/100))
    out["fiber_per_100kcal"]   = (out["fiber_g"]   / (kcal/100))
    out["protein_per_100kcal"] = (out["protein_g"] / (kcal/100))
    out["sodium_per_100kcal"]  = (out["sodium_mg"] / (kcal/100))

    # Ratios
    out["fiber_to_carbs"] = out["fiber_g"] / (out["carbs_g"] + 1e-6)
    out["sugar_to_carbs"] = out["sugar_g"] / (out["carbs_g"] + 1e-6)

    # Heuristic flags
    out["high_protein_flag"] = (out["protein_pct"] > 0.30).astype(int)
    out["high_fiber_flag"]   = (out["fiber_per_100kcal"] > 1.2).astype(int)
    out["high_sodium_flag"]  = (out["sodium_per_100kcal"] > 140).astype(int)

    # Categorical encodings
    # One-hot + explicit cooking flags
    out["is_fried"]   = (out["cooking_method"] == "Fried").astype(int)
    out["is_raw"]     = (out["cooking_method"] == "Raw").astype(int)
    out["is_steamed"] = (out["cooking_method"] == "Steamed").astype(int)
    cat_cols = ["cuisine", "meal_type", "diet_type", "cooking_method"]
    dummies = pd.get_dummies(out[cat_cols], prefix=cat_cols, drop_first=True, dtype=int)
    out = pd.concat([out.drop(columns=cat_cols), dummies], axis=1)

    # Optional winsorization of densities/ratios to curb extreme outliers
    to_clip = [
        "cals_per_100g","protein_100g","carbs_100g","fat_100g","fiber_100g","sugar_100g",
        "sugar_per_100kcal","fiber_per_100kcal","protein_per_100kcal","sodium_per_100kcal",
        "fiber_to_carbs","sugar_to_carbs"
    ]
    q_low, q_hi = clip_quantiles
    for c in to_clip:
        if c in out.columns:
            lo, hi = out[c].quantile(q_low), out[c].quantile(q_hi)
            out[c] = out[c].clip(lo, hi)

    # --- Drop any remaining obvious non-features if present
    # Keep label
    return out

# Usage
fe = engineer_meal_features(df)
y = fe["is_healthy"].astype(int).values
X = fe.drop(columns=["is_healthy"])

#### Numerical Feature Normalization

In [11]:
y = fe["is_healthy"].astype(int).values
X = fe.drop(columns=["is_healthy"]).copy()

# Identify binary/dummy columns vs continuous
# (binary = only 0/1 values; everything else treat as continuous)
bin_cols = [c for c in X.columns
            if pd.api.types.is_numeric_dtype(X[c]) and set(np.unique(X[c])).issubset({0,1})]
cont_cols = [c for c in X.columns if c not in bin_cols]

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, stratify=y, random_state=42
)

# Standardize continuous cols with TRAIN stats only
mu = X_train[cont_cols].mean()
sd = X_train[cont_cols].std().replace(0, 1)

X_train_s = X_train.copy()
X_test_s  = X_test.copy()
X_train_s[cont_cols] = (X_train[cont_cols] - mu) / sd
X_test_s[cont_cols]  = (X_test[cont_cols]  - mu) / sd

# Final arrays for models
Xtr = X_train_s.values.astype("float32")
Xte = X_test_s.values.astype("float32")
ytr = y_train.astype("float32")
yte = y_test.astype("float32")

print("Shapes:", Xtr.shape, Xte.shape)
print("Positives in train/test:", ytr.sum(), yte.sum())

Shapes: (1600, 55) (400, 55)
Positives in train/test: 150.0 37.0


### Correlation Analysis

In [12]:
label = "is_healthy"

# numeric columns only (so text like cuisine/meal_name is skipped)
num_cols = fe.select_dtypes(include=[np.number]).columns

# correlations w.r.t. the label
corr_with_label = (
    fe[num_cols]
    .corr(numeric_only=True)[label]        # correlation vector vs label
    .drop(labels=[label], errors="ignore") # remove self-correlation if present
    .dropna()
    .sort_values(ascending=False)
)

print("Correlation with is_healthy (descending):\n")
print(corr_with_label)

# Top-N by absolute correlation (useful when signs mix)
topN = 10
print(f"\nTop {topN} features by |correlation|:\n")
print(corr_with_label.abs().sort_values(ascending=False).head(topN))

Correlation with is_healthy (descending):

high_protein_flag         0.171357
carb_pct                  0.169994
fiber_per_100kcal         0.168588
protein_pct               0.165771
protein_per_100kcal       0.158617
sodium_per_100kcal        0.148253
high_sodium_flag          0.089673
high_fiber_flag           0.036841
meal_type_Snack           0.028691
is_quick_<=20min          0.026387
diet_type_Vegan           0.026104
cooking_method_Boiled     0.025710
diet_type_Paleo           0.024267
cooking_method_Roasted    0.021395
total_time_min            0.019633
diet_type_Keto            0.018192
cuisine_Indian            0.017928
cuisine_Thai              0.017208
cook_time_min             0.015366
sodium_mg                 0.014479
prep_time_min             0.013419
carbs_g                   0.013203
kcal_from_carbs           0.013203
protein_g                 0.012308
kcal_from_protein         0.012308
serving_size_g            0.011930
cuisine_Mediterranean     0.010768
fiber_g     

### Build the Neural Network Model
* Set-up Model Architecture
* Experiment with parameters:
  * `lrs = [2e-4, 5e-4, 1e-3, 2e-3]`
  * `bss = [16, 32, 64]`

In [13]:
tf.keras.utils.set_random_seed(42)

def build_model(input_dim, lr=1e-3, l2=1e-5, dropout=0.1):
    # L2 regularization to penalize very large weights (to prevent overfitting)
    # l2=1e-5 where each weight contributes a small penalty to the loss
    reg = keras.regularizers.l2(l2) if l2 else None
    m = keras.Sequential([
        keras.layers.Input(shape=(input_dim,)),
        keras.layers.Dense(64, activation="relu", kernel_regularizer=reg),
        keras.layers.Dropout(dropout),
        keras.layers.Dense(32, activation="relu", kernel_regularizer=reg),
        keras.layers.Dense(16, activation="relu", kernel_regularizer=reg),
        keras.layers.Dense(1, activation="sigmoid")
    ])
    m.compile(
        optimizer=keras.optimizers.Adam(learning_rate=lr),
        loss="binary_crossentropy",
        metrics=[keras.metrics.AUC(name="auc")]
    )
    return m

# Gives more weight to minority class during training
pos = int((ytr==1).sum()); neg = int((ytr==0).sum()); tot = len(ytr)
class_weight = {0: tot/(2*neg), 1: tot/(2*pos)}
print("Class weights:", class_weight)

lrs = [2e-4, 5e-4, 1e-3, 2e-3]
bss = [16, 32, 64]
pat = 10

results = []
for lr in lrs:
    for bs in bss:
        model = build_model(Xtr.shape[1], lr=lr, l2=1e-5, dropout=0.1)
        es = keras.callbacks.EarlyStopping(monitor="val_loss", patience=pat, restore_best_weights=True)
        model.fit(
            Xtr, ytr,
            validation_split=0.2,
            epochs=200,
            batch_size=bs,
            callbacks=[es],
            class_weight=class_weight,
            verbose=0
        )
        p = model.predict(Xte, verbose=0).ravel()
        yhat = (p >= 0.5).astype(int)

        acc = accuracy_score(yte, yhat)
        auc = roc_auc_score(yte, p)
        f1  = f1_score(yte, yhat)
        results.append((lr, bs, acc, auc, f1))
        print(f"LR={lr:<6} | BS={bs:<3} | Acc={acc:.3f}  AUC={auc:.3f}  F1={f1:.3f}")

best = max(results, key=lambda x: x[4])  # pick by F1; change to [3] for AUC
print("\nBest combo (by F1): LR=%.0e, Batch=%d | Acc=%.3f  AUC=%.3f  F1=%.3f"
      % (best[0], best[1], best[2], best[3], best[4]))

Class weights: {0: 0.5517241379310345, 1: 5.333333333333333}
LR=0.0002 | BS=16  | Acc=0.948  AUC=0.976  F1=0.741
LR=0.0002 | BS=32  | Acc=0.955  AUC=0.981  F1=0.775
LR=0.0002 | BS=64  | Acc=0.958  AUC=0.975  F1=0.779
LR=0.0005 | BS=16  | Acc=0.948  AUC=0.968  F1=0.734
LR=0.0005 | BS=32  | Acc=0.958  AUC=0.975  F1=0.779
LR=0.0005 | BS=64  | Acc=0.945  AUC=0.979  F1=0.738
LR=0.001  | BS=16  | Acc=0.948  AUC=0.976  F1=0.747
LR=0.001  | BS=32  | Acc=0.953  AUC=0.975  F1=0.753
LR=0.001  | BS=64  | Acc=0.963  AUC=0.984  F1=0.805
LR=0.002  | BS=16  | Acc=0.940  AUC=0.979  F1=0.721
LR=0.002  | BS=32  | Acc=0.950  AUC=0.977  F1=0.750
LR=0.002  | BS=64  | Acc=0.945  AUC=0.975  F1=0.718

Best combo (by F1): LR=1e-03, Batch=64 | Acc=0.963  AUC=0.984  F1=0.805


### Cross Fold Validation

- 5 folds with shuffle
- Per-fold standardization (prevents data leakage)
- Same architecture and hyperparameters across folds
- Class weights recalculated per fold

In [14]:
y = fe["is_healthy"].astype(int).values
X = fe.drop(columns=["is_healthy"]).copy()

# Use the best hyperparams
best_lr = globals().get("best_lr", 1e-3)   # default if not defined
best_bs = globals().get("best_bs", 16)     # default if not defined
l2 = 1e-5
dropout = 0.1
patience = 10
epochs = 200

# 5-fold CV
kf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
accs, aucs, f1s = [], [], []

fold = 0
for tr_idx, te_idx in kf.split(X, y):
    fold += 1

    X_tr_df = X.iloc[tr_idx].copy()
    X_te_df = X.iloc[te_idx].copy()
    y_tr = y[tr_idx]
    y_te = y[te_idx]

    # Scale continuous columns using TRAIN-FOLD stats
    mu = X_tr_df[cont_cols].mean()
    sd = X_tr_df[cont_cols].std().replace(0, 1)
    X_tr_df[cont_cols] = (X_tr_df[cont_cols] - mu) / sd
    X_te_df[cont_cols] = (X_te_df[cont_cols] - mu) / sd

    # To float32 arrays
    X_tr = X_tr_df.values.astype("float32")
    X_te = X_te_df.values.astype("float32")
    y_tr = y_tr.astype("float32")
    y_te = y_te.astype("float32")

    # Handle imbalance
    pos = int((y_tr == 1).sum()); neg = int((y_tr == 0).sum()); tot = len(y_tr)
    class_weight = {0: tot/(2*neg), 1: tot/(2*pos)}

    # Build & train
    tf.keras.utils.set_random_seed(42)
    model = build_model(X_tr.shape[1], lr=best_lr, l2=l2, dropout=dropout)
    es = keras.callbacks.EarlyStopping(monitor="val_loss", patience=patience, restore_best_weights=True)

    model.fit(
        X_tr, y_tr,
        validation_split=0.2,
        epochs=epochs,
        batch_size=best_bs,
        callbacks=[es],
        class_weight=class_weight,
        verbose=0
    )

    # Evaluate on the held-out fold
    p = model.predict(X_te, verbose=0).ravel()
    yhat = (p >= 0.5).astype(int)  # fixed 0.50 threshold for fair fold-to-fold comparison

    acc = accuracy_score(y_te, yhat)
    auc = roc_auc_score(y_te, p)
    f1  = f1_score(y_te, yhat)

    accs.append(acc); aucs.append(auc); f1s.append(f1)
    print(f"Fold {fold}: Acc={acc:.3f}  AUC={auc:.3f}  F1={f1:.3f}")

# Summary
print("\n5-Fold CV Summary (fixed 0.50 threshold):")
print("Accuracy : %.3f ± %.3f" % (np.mean(accs), np.std(accs)))
print("ROC-AUC  : %.3f ± %.3f" % (np.mean(aucs), np.std(aucs)))
print("F1       : %.3f ± %.3f" % (np.mean(f1s), np.std(f1s)))

Fold 1: Acc=0.960  AUC=0.983  F1=0.778
Fold 2: Acc=0.927  AUC=0.972  F1=0.695
Fold 3: Acc=0.950  AUC=0.981  F1=0.762
Fold 4: Acc=0.968  AUC=0.992  F1=0.835
Fold 5: Acc=0.968  AUC=0.987  F1=0.840

5-Fold CV Summary (fixed 0.50 threshold):
Accuracy : 0.955 ± 0.015
ROC-AUC  : 0.983 ± 0.007
F1       : 0.782 ± 0.053


The low standard deviations indicate a stable, reliable model performance!

### Final Model - Refit

In [15]:
# Refit final model with best hyperparams
best_lr, best_bs = 2e-3, 32
final_model = build_model(Xtr.shape[1], lr=best_lr, l2=1e-5, dropout=0.1)
es = keras.callbacks.EarlyStopping(monitor="val_loss", patience=10, restore_best_weights=True)
final_model.fit(
    Xtr, ytr, validation_split=0.2, epochs=200, batch_size=best_bs,
    callbacks=[es], class_weight=class_weight, verbose=0
)

# Threshold tuning on test set
# Get predicted probabilities
p = final_model.predict(Xte, verbose=0).ravel()

# Build precision-recall curve 
prec, rec, thr = precision_recall_curve(yte, p)

# Compute f1 score for each threshold
# F1 = 2 × (P × R)/(P + R)
# +1e-12 prevents division-by-zero errors
f1s = 2*prec*rec/(prec+rec+1e-12)

# Find the index where F1 is highest.
ix = f1s.argmax()
thr_best = thr[ix] if ix < len(thr) else 0.5

yhat50  = (p >= 0.50).astype(int)
yhatbest= (p >= thr_best).astype(int)

print(f"Best-F1 threshold: {thr_best:.3f}")
print("At 0.50  | Acc=%.3f  AUC=%.3f  F1=%.3f" % (accuracy_score(yte, yhat50),  roc_auc_score(yte, p), f1_score(yte, yhat50)))
print("At best  | Acc=%.3f  AUC=%.3f  F1=%.3f" % (accuracy_score(yte, yhatbest), roc_auc_score(yte, p), f1_score(yte, yhatbest)))
print("\nConfusion matrix @best:\n", confusion_matrix(yte, yhatbest))
print("\nClassification report @best:\n", classification_report(yte, yhatbest, digits=3))

Best-F1 threshold: 0.834
At 0.50  | Acc=0.948  AUC=0.975  F1=0.720
At best  | Acc=0.963  AUC=0.975  F1=0.776

Confusion matrix @best:
 [[359   4]
 [ 11  26]]

Classification report @best:
               precision    recall  f1-score   support

         0.0      0.970     0.989     0.980       363
         1.0      0.867     0.703     0.776        37

    accuracy                          0.963       400
   macro avg      0.918     0.846     0.878       400
weighted avg      0.961     0.963     0.961       400

